In [ ]:
import sys
sys.path.append("../")
import numpy as np
from qiskit import transpile
from qiskit_algorithms.optimizers import SPSA
from sanchez_ansatz import SanchezAnsatz
from experiments.experiment_module import ExperimentModule
from experiments.create_state import get_state
from experiments.util import get_state_vector
import matplotlib.pyplot as plt

%matplotlib inline

### Normal distribution

In [ ]:
interval = (0, 1) 

state_params = {"x_points": interval, "loc": 0.5, "scale": 0.6}

num_qubits = 8

x_points = np.linspace(*interval, 2**num_qubits)
normal_amplitudes = get_state(num_qubits, "normal", state_params)

sanchez = SanchezAnsatz(normal_amplitudes, eps=0.05)
init_params = sanchez.init_params
t_sanchez = transpile(sanchez, basis_gates=["u", "cx"])

em = ExperimentModule(t_sanchez, SPSA(maxiter=1000), normal_amplitudes, init_params,  device="GPU")

res = em.minimize("mse")

t_sanchez = t_sanchez.assign_parameters(res.x)

result_sv = get_state_vector(t_sanchez)

In [ ]:
plt.plot(x_points, normal_amplitudes)
plt.plot(x_points, result_sv)
plt.grid()
plt.show()

### Black-Scholes

In [ ]:
interval = (-10, 10)

state_params = {"x_points": interval, "K": 45, "c": 3}

num_qubits = 5
x_points = np.linspace(0, 2**num_qubits, 2**num_qubits)
black_scholes_state = get_state(num_qubits, "black-scholes", state_params)

plt.plot(x_points, black_scholes_state)
plt.show()
